# Different PSDs

We saw in the previous notebook just the psd.GammaPSD

The psd module also provide some other usefule PSDs:

In [ ]:
%pylab inline
from pytmatrix import tmatrix, scatter, radar, psd
from pytmatrix import tmatrix_aux as aux
from pytmatrix import refractive as ref
from pytmatrix import orientation as ori


#PSDs
Let's have a look to the provided PSDs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
diam = np.linspace(0.5,8.0,100)

# Exponential
exponential = psd.ExponentialPSD(N0=300.0,Lambda=1)

# Normalized Gamma
gamma = psd.GammaPSD(Nw=1.0e3,mu=4.0,D0=2.0)

# Unnormalized Gamma
Ugamma = psd.UnnormalizedGammaPSD(N0=1000.0,Lambda=2.,mu=1.2)

# Binned
bins = np.linspace(0.5,8.5,17)
bin_c = np.linspace(1,8,16)
binned = psd.BinnedPSD(bin_edges=bins,bin_psd=Ugamma(bin_c))

plt.bar(left=bins[0:-1],height=binned(bin_c),width=bins[1]-bins[0],color='green',label='binned')
plt.plot(diam,exponential(diam),linewidth=2,label='exp')
plt.plot(diam,gamma(diam),'r',linewidth=2,label='gamma')
plt.plot(diam,Ugamma(diam),'m',linewidth=2,label='UNgamma')
plt.xlabel('Diameter  [mm]')
plt.ylabel('Number concentration [m^{-3}]')
plt.legend()

# Complete example of radar simulation
### Preliminaries

In [ ]:
drops = tmatrix.Scatterer(wavelength=aux.wl_C,m=ref.m_w_10C[aux.wl_C])
drops.Kw_sqr = aux.K_w_sqr[aux.wl_C]
drops.or_pdf = ori.gaussian_pdf(std=60.0)
drops.orient = ori.orient_averaged_fixed
drops.psd_integrator = psd.PSDIntegrator()
drops.psd_integrator.D_max = 10.0
drops.psd_integrator.axis_ratio_func = lambda D: 1.0/aux.dsr_thurai_2007(D)

back = (120.0,60.0,0.0,180.0,0.0,0.0)
forw = (120.0,120.0,0.0,0.0, 0.0,0.0)
drops.psd_integrator.geometries = (back,forw)

In [ ]:
drops.psd_integrator.init_scatter_table(drops)

## Radar calculation

In [ ]:
drops.psd = gamma
from pytmatrix import radar
drops.set_geometry(back)
print 'Zh\t', radar.refl(drops), '\t[mm⁶/m³]'  # Radar quantities are in linear scale!!
print 'Zh\t',10*np.log10(radar.refl(drops)) ,'\t[dBZ]'
print 'Zv\t', radar.refl(drops,False), '\t[mm⁶/m³]'  # Radar quantities are in linear scale!!
print 'Zv\t',10*np.log10(radar.refl(drops,False)) ,'\t[dBZ]'
print 'Zdr\t',10*np.log10(radar.Zdr(drops)) ,'\t[dBZ]'
print 'rho_hv\t',radar.rho_hv(drops)
print 'del_hv\t',radar.delta_hv(drops), '\t[rad]'
print 'ldr\t',radar.ldr(drops) 
print 'LDR\t',10*np.log10(radar.ldr(drops)) ,'\t[dBZ]'
drops.set_geometry(forw)
print 'Kdp\t',radar.Kdp(drops) ,'\t[deg/km]'
print 'Ai\t',radar.Ai(drops) ,'\t[dB/km]'  # attenuation is already dBZ

#### Please do NOT take these as reference values for scattering calculations